In [4]:
import os
import json
import shelve

import warnings
warnings.filterwarnings("ignore")

import torch
import numpy as np

from prolog.prolog_generation import PrologData

from data.ag.action_genome import AG

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# Load dataset
root = '/data/Datasets/ag/'
subset_file = 'data/ag/subset_shelve'
train_ag = AG(root, no_img=True, split='train', subset_file=subset_file)
val_ag = AG(root, no_img=True, split='val', subset_file=subset_file)
test_ag = AG(root, no_img=True, split='test', subset_file=subset_file)
print(train_ag.verb_label_counts)
print(val_ag.verb_label_counts)
print(test_ag.verb_label_counts)

split: train length: 6388
split: val length: 798
split: test length: 799
[ 185  573   72   93  287  401  118   24  443  321   53    5  767   62
  219   76 2059  158  864  607  371  131  770  335   43  445  419   33
  135  274  166  475  113]
[ 20  78   3   8  48  49  14   5  54  45   8   3  82  10  28  12 253  32
 103  76  37  12 107  32   4  53  45   4  12  29   9  45  10]
[ 20  79  15  10  49  40  16   5  60  37  13   0  96  11  21  15 256  21
 124  93  43  12 106  45   7  57  39   5  16  34  22  69  15]


In [33]:
%autoreload
# Load prolog data
train_pd = PrologData('prolog', 'ag', train_ag, train_ag.object_classes, train_ag.relationship_classes, train_ag.verb_classes, model=None, split='train')

val_pd = PrologData('prolog', 'ag', val_ag, val_ag.object_classes, val_ag.relationship_classes, val_ag.verb_classes, model=None, split='val')

test_pd = PrologData('prolog', 'ag', test_ag, test_ag.object_classes, test_ag.relationship_classes, test_ag.verb_classes, model=None, split='test')

train_pd.write_bk()
train_pd.init_general_bias()

val_pd.write_bk()
test_pd.write_bk()

ag
ag
ag


In [35]:
def exp_curve(b,x):
    return 1-np.exp(-b*x)

dataset = val_ag
for verb_idx, verb_name in enumerate(dataset.verb_classes):

    freq = dataset.verb_label_counts[verb_idx]
    ratio = freq/len(dataset)
    print(verb_name, ratio)

    #keeps negatives according to the frequency of the verb
    train_pd.write_verb(verb_name, keep_prob=exp_curve(4, ratio)) 

awaken 0.015037593984962405
close 0.058646616541353384
cook 0.002255639097744361
dress 0.006015037593984963
drink 0.03609022556390978
eat 0.03684210526315789
fix 0.010526315789473684
grasp 0.0037593984962406013
hold 0.0406015037593985
laugh 0.03383458646616541
lie 0.006015037593984963
make 0.002255639097744361
open 0.061654135338345864
photograph 0.007518796992481203
play 0.021052631578947368
pour 0.009022556390977444
put 0.19022556390977444
run 0.02406015037593985
sit 0.0774436090225564
smile 0.05714285714285714
sneeze 0.027819548872180452
snuggle 0.009022556390977444
stand 0.08045112781954887
take 0.02406015037593985
talk 0.0030075187969924814
throw 0.03984962406015038
tidy 0.03383458646616541
turn 0.0030075187969924814
undress 0.009022556390977444
walk 0.02180451127819549
wash 0.006766917293233083
watch 0.03383458646616541
work 0.007518796992481203
